In [393]:
from requests import get
from bs4 import BeautifulSoup
import os
import re
import path
import json

In [51]:
url = 'https://codeup.com/competitor-bootcamps-are-closing-is-the-model-in-danger/'
url_list = ['https://codeup.com/data-science-vs-data-analytics-whats-the-difference/',
        'https://codeup.com/codeups-data-science-career-accelerator-is-here/',
        'https://codeup.com/data-science-myths/',
        'https://codeup.com/10-tips-to-crush-it-at-the-sa-tech-job-fair/',
        'https://codeup.com/competitor-bootcamps-are-closing-is-the-model-in-danger/']

headers = {'User-Agent': 'Codeup Ada Data Science'}
response = get(url, headers=headers)

In [5]:
print(response.text[:400])

<!DOCTYPE html>
<html lang="en-US" prefix="og: http://ogp.me/ns#" >
<head>
		<meta charset="UTF-8" /><meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, user-scalable=0" /><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" /><meta name="format-detection" content="telephone=no"><title>Codeup’s Data Science Career Accelerator is Here


In [6]:
soup = BeautifulSoup(response.content, 'html.parser')

In [17]:
article = soup.find('div', class_='mk-single-content')
title = soup.find(class_='page-title')
author = soup.find(class_='mk-blog-author')
date = soup.find(class_='mk-post-date')
category = soup.find(class_='mk-post-cat')

In [19]:
print(author.text)

By Yumi Jeon 


In [14]:
with open('article.txt', 'w') as f:
    f.write(article.text)

In [378]:
def get_url_list():
    urls = []
    headers = {'User-Agent': 'Codeup Ada Data Science'}
    url = 'https://codeup.com/blog/'
    response = get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    max_page = soup.find(class_='pagination-max-pages').text
    max_page = int(max_page) + 1
    for i in range(1,max_page):
        url = 'https://codeup.com/blog/page/' + str(i) + '/'
        print(f'traversing url: {url}')
        response = get(url, headers=headers)
        soup = BeautifulSoup(response.text)
        for h in soup.find_all('h3'):
            a = h.find('a')
            urls.append(a.attrs['href'])
    print(len(urls))
    return urls

In [388]:
def get_all_blogs(urls):
    return [get_blog_text(url) for url in urls]

def get_blog_text(url):
    url = url
    headers = {'User-Agent': 'Codeup Ada Data Science'}
    response = get(url, headers=headers)
    soup = BeautifulSoup(response.text)
    article = soup.find('div', class_='mk-single-content')
    title = soup.find(class_='page-title')
    author = soup.find(class_='mk-blog-author')
    date = soup.find(class_='mk-post-date')
    category = soup.find(class_='mk-post-cat')
    
    title2 = re.findall(r'\/([^\/]+)\/?$', url)[0]
    
    print(f'writing: {title2}')
    with open('/Users/orion/Python/Methodologies/nlp/docs/' + title2 + '.txt', 'w') as f:
        content = title2 + author.text + date.text + category.text + article.text
        f.write(content)
    

    return {
            'Title': title2,
            'Author': author.text,
            'Date': date.text,
            'Category': category.text,
            'Article': article.text}

In [391]:
def get_blog_articles(use_cache=True):
    if use_cache and os.path.exists('/docs/codeup_blog_articles.json'):
        articles = json.load(open('/docs/codeup_blog_articles.json'))
    else:
        articles = get_all_blogs(get_url_list())
        json.dump(articles, open('/Users/orion/Python/Methodologies/nlp/docs/codeup_blog_articles.json', 'w'))
    return articles

In [392]:
get_blog_articles()

traversing url: https://codeup.com/blog/page/1/
traversing url: https://codeup.com/blog/page/2/
traversing url: https://codeup.com/blog/page/3/
traversing url: https://codeup.com/blog/page/4/
traversing url: https://codeup.com/blog/page/5/
traversing url: https://codeup.com/blog/page/6/
traversing url: https://codeup.com/blog/page/7/
63
writing: 5-common-excuses-keeping-you-from-breaking-into-the-tech-field
writing: why-san-antonio-has-more-than-tacos-to-offer
writing: everyday-encounters-with-data-science
writing: path-codeup-coding-bootcamps
writing: codeup-student-check-in-month-3
writing: styling-hair-stying-interfaces
writing: from-cooking-steaks-to-cooking-code
writing: dont-be-scared-of-coding
writing: debugging-with-codey-the-rubber-duck
writing: from-the-service-industry-to-software-development
writing: codeup-student-check-in-month-2
writing: codeups-data-science-career-accelerator-is-here
writing: from-a-recent-college-grad-to-software-developer
writing: 5-things-codeup-does

[{'Title': '5-common-excuses-keeping-you-from-breaking-into-the-tech-field',
  'Author': 'By Joyce Ling ',
  'Date': '\n\t\t\t\xa0Posted April 26, 2019\n',
  'Category': '\xa0In Uncategorized',
  'Article': '\nJust a few months before starting at Codeup in the Redwood cohort, I was sitting in the football stadium at the University of Colorado at Boulder, pondering what I would do after graduation. The commencement speaker that year was Kate Fagan, a sports reporter and commentator at ESPN. In her speech, something she said stuck out to me: “Try replacing ‘should’ with ‘want’ and, as frequently as you are able, make decisions with that rubric. Life is best when your ‘should’ and your ‘want’ are aligned.” Sitting there in that stadium, I realized that I knew exactly what I should be doing after graduating, which was applying to attend graduate school for the next five years. But the actual truth was, I didn’t know what I truly wanted. Did I really want to jump into something for five yea

In [171]:
headers = {'User-Agent': 'Codeup Ada Data Science'}
url = 'https://codeup.com/blog/'

response = get(url, headers=headers)
soup = BeautifulSoup(response.text)

urls = []
for h in soup.find_all('h3'):
    a = h.find('a')
    urls.append(a.attrs['href'])

### News Articles

We will now be scraping text data from [inshorts](https://inshorts.com/), a website that provides a brief overview of many different topics.

Write a function that scrapes the news articles for the following topics:

* business
* sports
* technology
* entertainment

The end product of this should be a function named get_news_articles that returns a list of dictionaries, where each dictionary has this shape:

```
{
    'title': 'The article title',
    'content': 'The article content',
    'category': 'business' # for example
}
```

In [180]:
topics = ['business', 'sports', 'technology', 'entertainment']
url = 'https://inshorts.com/en/read/business'

In [181]:
soup = BeautifulSoup(response.content, 'html.parser')
response = get(url, headers=headers)

In [231]:
title = soup.find(class_='news-card-title').find('a').text.replace('\n', '')
content = soup.find(class_='news-card-content').find('div').text
category = re.findall(r'\/([^\/]+)\/?$', url)[0]

In [232]:
card = soup.find_all(class_='news-card')

In [306]:
biglist = []

topics = ['business', 'sports', 'technology', 'entertainment']

def get_news_articles(topics, refresh=False):
    if refresh == True:
        initiate_news_scrape()
    for t in topics:
        url = url = 'https://inshorts.com/en/read/' + t
        print(f'Reading from: {url}')
        get_articles(url)
    with open('docs/inshorts-news.txt', 'w') as f:
        f.write(str(biglist))

def get_articles(url):
        response = get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        for h in soup.find_all(class_='news-card'):
            title = h.find(class_='news-card-title').find('a').text.replace('\n', '')
            content = h.find(class_='news-card-content').find('div').text
            category = re.findall(r'\/([^\/]+)\/?$', url)[0]
            inshorts = {'Title': title,
                        'Category': category,
                        'Content': content}
            biglist.append(inshorts)
            
def initiate_news_scrape():
    if os.path.exists('docs/inshorts-news.txt'):
        with open('docs/inshorts-news.txt') as f:
            return f.read()
    else:
        return

In [307]:
get_news_articles(topics)

Reading from: https://inshorts.com/en/read/business
Reading from: https://inshorts.com/en/read/sports
Reading from: https://inshorts.com/en/read/technology
Reading from: https://inshorts.com/en/read/entertainment
